In [34]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import os

# 경로 설정
data_path = '/opt/ml/uplus/data'
saved_path = '../saved'
output_path = '../submission'

# 필요한 데이터 불러오기
profile_df = pd.read_csv(os.path.join(data_path, 'profile_data.csv'), encoding='utf-8')
history_df = pd.read_csv(os.path.join(data_path, 'history_data.csv'), encoding='utf-8')

In [35]:
# 같은 시간에 시청한 이력 제거
data = history_df[['profile_id', 'log_time', 'album_id']].drop_duplicates(subset=['profile_id', 'album_id', 'log_time']).sort_values(by = ['profile_id', 'log_time']).reset_index(drop = True)

# log_time 컬럼은 필요없으므로 제거
data = data.drop('log_time', axis=1)

In [36]:
# profile_df에서 필요한 컬럼만 가져오기
age_df = profile_df[['profile_id', 'age']]

In [37]:
# age 그룹화(1~3세, 4~5세, 6~7세, 8~9세, 10~13세)
# 실험결과 이렇게 나누는 것이 가장 최적의 그룹 구분법이였습니다.
age_df['age'] = age_df['age'].replace(13, 10)
age_df['age'] = age_df['age'].replace(12, 10)
age_df['age'] = age_df['age'].replace(11, 10)
age_df['age'] = age_df['age'].replace(3, 1)
age_df['age'] = age_df['age'].replace(2, 1)
age_df['age'] = age_df['age'].replace(4, 5)
age_df['age'] = age_df['age'].replace(6, 7)
age_df['age'] = age_df['age'].replace(8, 9)

In [38]:
# user별로 가장 많이 본 순서대로 아이템 정렬(rulebase)
rulebase_user = data.groupby(['profile_id']).apply(lambda x : x['album_id'].value_counts()).reset_index()
rulebase_user = rulebase_user.groupby('profile_id')['level_1'].unique().to_frame().reset_index()

# 나이 정보 추가(위에서 생성한 데이터 프레임에 나이 정보 추가)
rulebase_user = pd.merge(rulebase_user, age_df, on = 'profile_id', how = 'left')

In [39]:
# 나이 정보 추가(history_df를 적절하게 가공한 데이터 프레임에 나이 정보 추가)
data_df = pd.merge(data, age_df, on = 'profile_id', how = 'left')

# 나이별로 가장 많이 본 순서대로 아이템 정렬
rulebase_age = data_df.groupby(['age']).apply(lambda x : x['album_id'].value_counts()).reset_index()
rulebase_age = rulebase_age.groupby('age')['level_1'].unique().to_frame().reset_index()

In [40]:
# user별 데이터와 나이별 데이터를 merge
result_df = pd.merge(rulebase_user, rulebase_age, on = 'age', how = 'left')

In [41]:
# 데이터 결합하는 함수
def funs(x):
    # user별 데이터와 나이별 데이터를 하나의 리스트로 합쳐 줌
    tem = list(x['level_1_x']) + list(x['level_1_y'])
    result = []
    # 리스트의 아이템 개수가 25개가 될 때까지, 중복 없이 리스트에 아이템을 추가
    for i in tem:
        if i not in result:
            result.append(i)
        if len(result) == 25:
            break
    return result

# 유저가 25개 이하의 아이템을 소비한 경우, 그 뒤에 나이별로 가장 많이 본 순서대로 정렬한 아이템을 붙여주는 코드
result_df['predicted_list'] = result_df.apply(lambda x : list(x['level_1_x'][:25]) if len(x['level_1_x']) >= 25 else funs(x), axis = 1)

In [42]:
# 데이터를 파일로 저장
result_df[['profile_id', 'predicted_list']].to_csv(os.path.join(output_path, 'submission_rulebase_age.csv'), index = False)